In [6]:
import os
import json
import numpy as np
import pandas as pd
from pathlib import Path
import librosa
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor
from functools import partial
from transformers import AutoProcessor, Wav2Vec2ForSequenceClassification
from torch.optim import AdamW

g:\vseross\two-step_A-task\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
TRAIN_DIR = 'train_opus/audio'
TEST_DIR = 'test_opus/audio'
WORD_BOUNDS = 'train_opus/word_bounds.json'
SAMPLE_RATE = 16000

In [ ]:
with open(WORD_BOUNDS, 'r') as f:
    word_bounds = json.load(f)

all_train_files = [f for f in Path(TRAIN_DIR).glob('*.opus') if not f.name.startswith('._')]
test_files = [f for f in Path(TEST_DIR).glob('*.opus') if not f.name.startswith('._')]

SUBSET_SIZE = 0.3
train_files = all_train_files[:int(len(all_train_files) * SUBSET_SIZE)]

train_ids = [f.stem for f in train_files]
test_ids = [f.stem for f in test_files]

train_labels = {file_id: 1 if file_id in word_bounds else 0 for file_id in train_ids}

SAMPLE_RATE = 16000

print(f'Train files: {len(train_files)} ({SUBSET_SIZE*100:.0f}% от {len(all_train_files)})')
print(f'Test files: {len(test_files)} (100%)')

Train files: 90000 (100% от 90000)
Test files: 27000 (100%)


In [25]:
# Загружаем train
train0 = pd.read_csv('dataset/train_emb_0.csv').drop(columns='label')
train0.columns = ['id'] + [x+"_0" for x in train0.columns[1:]]

train1 = pd.read_csv('dataset/train_emb_1.csv').drop(columns='label')
train1.columns = ['id'] + [x+"_1" for x in train1.columns[1:]]

train2 = pd.read_csv('dataset/train_emb_2.csv').drop(columns='label')
train2.columns = ['id'] + [x+"_2" for x in train2.columns[1:]]

train = train0.merge(train1, on='id').merge(train2, on='id')
train['label'] = train['id'].map(train_labels)

In [68]:
class AudioDataset(Dataset):
    def __init__(self, data: pd.DataFrame, labels=None):
        self.data = data 
        self.labels = labels 

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx].values 
        row = torch.tensor(row, dtype=torch.float32)
       
        if self.labels is not None:
            label = torch.tensor(self.labels[idx], dtype=torch.long)
            return row, label
        else:
            return row

In [51]:
col_x = [x for x in train.columns if 'emb' in x]

X = train[col_x]
y = train['label'].values 

In [52]:
train_tr, val_tr, train_lbs, val_lbls = train_test_split(
    X, y, test_size=0.15, random_state=42, stratify=y
)

train_dataset = AudioDataset(train_tr, train_lbs)
val_dataset = AudioDataset(val_tr, val_lbls)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False, num_workers=0)

In [61]:
class AudioClassifier(nn.Module):
    def __init__(self, input_dim, num_classes=10):
        super(AudioClassifier, self).__init__()
        self.fc = nn.Linear(input_dim, num_classes)
        
    def forward(self, x):
        x = self.fc(x)
        return x


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = AudioClassifier(input_dim=len(col_x), num_classes=1)
model = model.to(device)

In [62]:
def train_epoch(model, loader, optimizer, criterion):
    model.train()
    total_loss = 0.0
    correct = 0
    total = 0
    for features, labels in tqdm(loader, desc='Training'):
        features = features.to(device)
        labels = labels.to(device).float().unsqueeze(1)
        optimizer.zero_grad()
        outputs = model(features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        preds = (outputs > 0).long().squeeze()
        correct += (preds == labels.squeeze().long()).sum().item()
        total += labels.size(0)
    return total_loss / len(loader), correct / total

def validate(model, loader, criterion):
    model.eval()
    total_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for features, labels in tqdm(loader, desc='Validation'):
            features = features.to(device)
            labels = labels.to(device).float().unsqueeze(1)
            outputs = model(features)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
            preds = (outputs > 0).long().squeeze()
            correct += (preds == labels.squeeze().long()).sum().item()
            total += labels.size(0)
    return total_loss / len(loader), correct / total

In [ ]:
optimizer = AdamW(model.parameters(), lr=1e-4)
criterion = nn.BCEWithLogitsLoss()

num_epochs = 10
best_acc = 0.0

for epoch in range(num_epochs):
    print(f'\nEpoch {epoch+1}/{num_epochs}')
    train_loss, train_acc = train_epoch(model, train_loader, optimizer, criterion)
    val_loss, val_acc = validate(model, val_loader, criterion)
    print(f'Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}')
    print(f'Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}')
    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(model.state_dict(), f'best_model_{epoch}.pth')
        print('Model saved!')

In [64]:
# Загружаем train
test0 = pd.read_csv('dataset/test_emb_0.csv').drop(columns='label')
test0.columns = ['id'] + [x+"_0" for x in train0.columns[1:]]

test1 = pd.read_csv('dataset/test_emb_1.csv').drop(columns='label')
test1.columns = ['id'] + [x+"_1" for x in test1.columns[1:]]

test2 = pd.read_csv('dataset/test_emb_2.csv').drop(columns='label')
test2.columns = ['id'] + [x+"_2" for x in test2.columns[1:]]

test = test0.merge(test1, on='id').merge(test2, on='id')

In [71]:
model.load_state_dict(torch.load('best_model_4.pth', map_location=device))
model.eval()

test_features = test.drop(columns='id')
test_dataset = AudioDataset(test_features)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

predictions = []
with torch.no_grad():
    for features in tqdm(test_loader, desc='Predicting'):
        features = features.to(device)
        outputs = model(features)
        preds = (outputs > 0).long().squeeze().cpu().numpy().tolist()
        predictions.extend(preds)

submission = pd.DataFrame({
    'id': test['id'],
    'label': predictions
})

submission.to_csv('submission.csv', index=False)
print('Submission saved!')

C:\Users\djadc\AppData\Local\Temp\ipykernel_20428\2356710224.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_model_4.pth', map_loc

Submission saved!
